# 非線形・交差項・二乗項

線型モデルは制約的に感じられるかもしれませんが，二乗や自然対数`ln(x)`の変換を行うことで， xの非線形の効果を捉えることも可能です

## 自然対数を用いた非線形モデルの推定

変数の自然対数をとって推定に用いることもよく行われます。

例えば、コブダグラス型の生産関数は通常、労働$L$と資本$K$を用いてアウトプット$Y$が算出されるテクノロジーとして$Y=L^aK^b$と、定式化されます。この場合、このまま生産関数を推定しようとすると非線形モデルとなりますが、両辺とも自然対数をとって$LnY=alnL+blnK$と変換することで線型モデルとして扱うことができます。

また、自然対数をとると被説明変数や説明変数の単位（円、千円、時間、分など ）によらず単位の異なるデータを用いた推定結果の比較や弾性値（弾力性）の推定が容易になります。両辺が自然対数をとった$lnY=\beta_0+\beta_1 ln X+ u$という式ではパラメータ$\beta_1$は説明変数Xが1％変化した時に被説明変数yが何％変化すると解釈することができます。

傾きパラメータの解釈
| 被説明変数 | 説明変数 |解釈                                          | 
| ---------- | -------- | :-------------------------------------------- | 
| $Y$          | $X$        | $X$が１単位増えた時$Y$が$\beta_1$単位増える         | 
| $lnY$       | $X$        | $X$が１単位増えた時$Y$が$100 \times \beta_1$%増える | 
| $Y$          | $lnX$     | $X$が1%増えた時$Y$が$\beta_1 / 100$ 単位増える      | 
| $lnY$       | $lnX$     | $X$が1%増えた時$Y$が$\beta_1$%増える      | 

## ダミー変数（Dummy variable）
性別や年齢層、職業カテゴリなどの連続値以外の質的データやカテゴリデータを用いて、その差が被説明変数に与える影響を調べたい際には、0または1の値を取る<strong>ダミー変数</strong>を用います。

例えば、高校卒業をしているかしていないかが賃金水準に与える影響を調べたい際には、高校を卒業していれば`1`それ以外は`0`を持つダミー変数を用います。ダミー変数はこれまでに扱った連続値と同様にモデルの中に入れて推定を行うことができます。

ダミー変数を用いる際には基準カテゴリが必要となります。例えば、卒業高校に関するカテゴリ変数で、公立高校卒業、私立高校卒業、高等専門学校卒業、それ以外（高校卒業未満を含む）という4つのカテゴリを持つ変数をモデルに入れる場合、公立高校卒業者だけ1をとるダミー、私立高校卒業者だけ1をとるダミー、高等専門学校卒業者だけ1をとるダミー、それ以外（高校卒業未満を含む）だけ1をとるダミーの4つのダミーを作ることが可能ですが、実際には1つの分類を<strong>基準カテゴリ（ベース）</strong>と設定して、そのベースを除いたダミー変数のみをモデルに入れます。つまり、カテゴリの数-1このダミー変数をモデルに入れます。これは多重共線性の問題を避けるためです。


例として、賃金に勤務年数が与える影響を考えます。
IT企業での賃金の関するデータを用いて、学歴（E）が賃金(S)に与える影響を見ていきます。
学歴(E)は 1=学部卒, 2=修士卒, 3=博士卒の値を格納していますが、ここでは、簡単のため、学歴（E）を学部卒なら0、修士卒以上なら1とする2値の新しいダミー変数をつくって進めます。

In [ ]:
from urllib.request import urlopen
import numpy as np
from statsmodels.formula.api import ols
import pandas as pd

> 本項の説明は、次のページのデータを用いています。一部説明も参考にしています。https://web.stanford.edu/class/stats191/

In [ ]:
url = 'http://stats191.stanford.edu/data/salary.table'
fh = urlopen(url)
salary_df = pd.read_table(fh, dtype={'S':int, 'X':int, 'E':int, 'M':int})
salary_df.head(2)

In [ ]:
# 学部卒なら0、修士卒以上なら1を持つダミー変数EHを作る
salary_df.loc[:,'EH'] = salary_df['E'].replace({1:0, 2:1, 3:1})
# 基本統計量を確認
salary_df['EH'].describe()

以下の単回帰式を求めます。

$$S_i=\beta_0+\beta_1EH_{i1}+\epsilon_i$$

### statsmodelsを用いたダミー変数の生成
`statsmodels`では、`C()`でカテゴリー変数から自動的にダミー変数に割り当てます。また、`Treatment()`を指定することで基準カテゴリ（ベース）を指定することができます。

In [ ]:
simple_formula = 'S ~ C(EH, Treatment(0))'
# Treatmentはデフォルトでは最も若い値が基準カテゴリとなるため、
# ここでは、以下の式でも同じ結果が得られます。
# simple_formula = 'S ~ C(EH)'としても同じ結果になります。
simple_res = ols(simple_formula, data = salary_df).fit()
print(simple_res.summary())

上で得られた結果の中で、`C(EH, Treatment(0))[T.1]`の`coef`を確認します。
`EH=1`（修士卒以上）の従業員は基準カテゴリである学部卒の従業員に比べて、賃金は$3347.5000平均で高いことがわかります（ただし、この分析では他の要因を考慮していないので留意が必要です）。

### pandasを用いたダミー変数の生成
次に参考までに、`C()`を用いないで、ダミー変数を先に`pandas`などで生成してから式に入れる方法でも実行してみましょう。

ここでは、`pandas`の`pd.get_dummies()`を用いてダミー変数をテーブルデータに新たな変数として追加してから式に渡す方法を紹介します。
3つの値（1=学部卒, 2=修士卒, 3=博士卒）を持つ`E`からダミー変数を作ります。

In [ ]:
dummies = pd.get_dummies(salary_df['E'], prefix='E', prefix_sep='')
salary_df = salary_df.join(dummies)
# 最初の２行を表示
salary_df.head(2)

In [ ]:
# ダミー変数はカテゴリーの数-1の変数を式に入れ、入れなかった変数を基準カテゴリとします
simple_formula = 'S ~ E2 + E3'
simple_res = ols(simple_formula, data = salary_df).fit()
print(simple_res.summary())


## 交差項（Interaction term）

ある変数が別の変数の値によって異なる影響を与えることを確認する際等に交差項を使います。

例えば、上の分析の例では、学歴が修士卒以上の従業員と学部卒の従業員の間で、マネージャーであることが賃金に与える影響の違いを捉えることができます。


上の例と同じ賃金に関するデータを用いて、交差項を見ていきます
賃金(S)と、学歴(EH, 0=学部卒, 1=修士/博士卒)、マネージャーかどうか（M, 1=management, 0= not management）の関係性を見ていきます。


まず、交差項を用いた分析をする前に、交差項を用いない以下の通常の重回帰分析を考えます。

$$S_i=\beta_0+\beta_1EH_{i1}+\beta_2M_{i1}+\epsilon_i$$

上の式で、$EH_i1$は修士・博士卒（EH＝1）であれば1それ以外であれば0を取るダミー変数です。


In [ ]:
formula = 'S ~ C(M, Treatment(0)) + C(EH, Treatment(0))'
# formula = 'S ~ C(M) + C(EH)' と書いてもここでは同じ結果が得られます。
lm = ols(formula, salary_df).fit()
print(lm.summary())

上の例のように交差項がない場合は、変数（マネージャーであること）のカテゴリー間の平均値の差は、他の変数（上の例の場合学歴）に関係なく同じであると仮定しています。

交差項を用いる場合は、変数カテゴリー（上の例ではマネージャであること）の平均値の差がが他の変数（上の例の場合は学部卒か修士・博士卒か）によって異なることを確認することができます。

交差項の係数の解釈では、交差項に入れた２つの変数に当てはまらい場合の平均値に比べて、追加で平均値に差をで与えると考えることができます。


次に、学歴($EH$)とマネージャーかどうかのダミー変数($M$)の交差項を考えます。

$$S_i=\beta_0+\beta_1EH_{i1}+\beta_2{M_i1}+\beta_3EH_{i1}M_{i1}+\epsilon_i$$

ここでは、2つのサブモデルを考えることができます。

* マネージャーでない場合($M=0$): $S_i=\beta_0+\beta_1{EH_{i1}}+\epsilon_i$
* マネージャーである場合($M=1$): $S_i=(\beta_0+\beta_2)+(\beta_1+\beta_3){EH_{i1}}+\epsilon_i$

パラメーターや係数の解釈例としては、以下のようになります。
* $\beta_0$：学部卒（$EH=0$）でマネージャではない場合（$M=0$）の平均賃金
* $\beta_1$：マネージャでない場合に（$M=0$）、修士・博士卒（$EH=1$）が賃金に与える効果
* $(\beta_0+\beta_2)$：学部卒の場合（$EH=0$）の場合のマネージャーであること際の（$M=1$）平均賃金
* $(\beta_1+\beta_3)$：マネージャーである場合（$M=1$）で修士・博士卒である場合（$EH=1$）の平均賃金

`statsmodels`では`:`を用いて交差項を表現できます。`C(M):C(EH)`で$\beta_3EH_{i1}M_{i1}$をつくります。

In [ ]:
interaction_formula = "S ~ C(EH) + C(M) + C(EH):C(M)"
interaction_ols= ols(interaction_formula, data=salary_df)
interaction_res = interaction_ols.fit()
print(interaction_res.summary())

得られた結果のうち交差項`C(M)[T.1]:C(EH)[T.1]`の係数`coef`と、そのt値・p値を確認します。
`coef`は7321.7634で、t値が十分に大きい（p値も十分に小さい）値であることから、マネージャーであることが賃金に与える影響は学歴（修士号・博士号の有無）によって効果が異なることが確認できます。

解釈としては、学部卒・非マネージャー職より、修士・博士号取得者は、マネージャになると平均で$7322高い賃金を得ている。


## 二乗項（Quadratic term）

賃金の上昇は若い時は経験年数が1年伸びると大きいものの、歳をとるにつれて伸びが緩やかになっていく場合もあるかもしれません。こうした変化を考慮するための１つの方法として、高次項を含む重回帰分析モデルを考えることができます。

$$S_i=\beta_0+\beta_1X+\beta_2X^2+\beta_3EH_{1i}+\beta_5EH_{1i}M_{i}+\beta_6M_{i}+\epsilon_i$$

statsmodels.from_formulaで二乗を回帰式に入れる場合は`I()`を直接fomula内に書くことができます。


In [ ]:
quadr_formula = "S ~ X + I(X**2) + C(EH) + C(M) + C(EH):C(M)"
quadr_ols= ols(quadr_formula, data=salary_df)
quadr_res = quadr_ols.fit()
print(quadr_res.summary())

得られた結果のうち`I(X ** 2)`が経験年数の二乗項の係数です。t値が小さくp値も大きいので、経験年数が二条項としての効果があるとは言えない（経験年数が増えるほど経験年数が浅い人に比べて、経験年数1年増加に対する効果は小さくなるとは言えない）と解釈できます。
